# Lab: Transfer Learning - Getting Started
We will explore transfer learning concepts

### Runtime
~ 15 minutes

### Note
Here we are dealing with real world images.  Processing them will required a lot of compute power.  
If you have access to, switch to **GPU** as run time!

**Instructor** walk through this lab step-by-step and explain

### References
- https://www.tensorflow.org/tutorials/images/transfer_learning
- [lmorony-dlaicourse](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%206%20-%20Lesson%203%20-%20Notebook.ipynb)


In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
print ('tensorflow version :', tf.__version__)
tf.config.experimental.list_physical_devices()

tensorflow version : 2.2.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

## TF-GPU Config
The following cell sets TF properties to run on GPU

In [2]:
## This block is to tweak TF running on GPU
## You may comment this out, if you are not using GPU

## ---- start Memory setting ----
## Ask TF not to allocate all GPU memory at once.. allocate as needed
## Without this the execution will fail with "failed to initialize algorithm" error

from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



### Custom Model Summary Function
This function only prints out the essentials

In [9]:
def compact_summary_helper(line):
    matches = ['Model:', 'Total params:', 'Trainable params:', 'Non-trainable params:']
    if any(x in line for x in matches):
        print("*", line)

def print_model_summary_compact(model):
    model.summary(print_fn=compact_summary_helper)
    # print ("* model name:", model.name)
    print ("* # layers: ", len(model.layers))
    


## Step 1 - Download MobileNetv2 Model
Here we will examine MobileNetv2 model.  MobileNetv2 was developed at Google.  It is pre-trained on the ImageNet dataset consisting of 1.4 Million images and 1000 classes.

- paper: [MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/abs/1704.04861)
- [github](https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/README.md)
- [Tensorflow model page](https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2)
- [Tensorflow transfer learning guide](https://www.tensorflow.org/tutorials/images/transfer_learning)

When downloading this model, the image dimension should be (224,224,3) to match Imagenet dimensions.

We are also initializing the model with the weights of 'imagenet'

#### TODO
- Notice how many layers we have :-) 
- Look at the final `prediction` layer.  Why does it have 1000 neurons?
- Look at `total parameters` and `total trainable parameters`

In [4]:
mobilenetv2_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), 
                                               include_top = True,
                                               weights = 'imagenet')

print ("# layers: ", len(mobilenetv2_model.layers))

## Warning: large output!
mobilenetv2_model.summary()

# layers:  157
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
________________________________________________________________

## Step 2 - Try Compact Summary
Let's try our print function

In [10]:
print_model_summary_compact(mobilenetv2_model)

* Model: "mobilenetv2_1.00_224"
* Total params: 3,538,984
* Trainable params: 3,504,872
* Non-trainable params: 34,112
* # layers:  157


In [23]:
## Let's find the size of downloaded model
! du -skh ~/.keras/models/*

92M	/home/sujee/.keras/models/inception_v3_weights_tf_dim_ordering_tf_kernels.h5
9.0M	/home/sujee/.keras/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_160_no_top.h5
14M	/home/sujee/.keras/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5
9.0M	/home/sujee/.keras/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5


## Step 3 -  Freeze Layers
We are going to freeze the model.  
See how many traininable parameters we  have now.  Compare with previous number.

In [11]:
mobilenetv2_model.trainable = False
print_model_summary_compact(mobilenetv2_model)

* Model: "mobilenetv2_1.00_224"
* Total params: 3,538,984
* Trainable params: 0
* Non-trainable params: 3,538,984
* # layers:  157


## Step 4 - Load Just the Base Model
The last layer of MobileNet is for predicting 1000 classes of ImageNet.  
For our purpose we don't need that.  
So we are going to just load the baes model.  This is achieve by specifying `include_top=False`  
Since we are not using the last layer, we can specify a custom image size other than (224,224,3)

### TODO 
- Compare number of layers of full model and base model
- Also compare number of trainable params of both models

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(160,160,3),
                                               include_top=False,
                                               weights='imagenet')
print ("# layers: ", len(base_model.layers))
base_model.summary()

# layers:  155
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
________________________________________________________________

In [14]:
print_model_summary_compact(base_model)

* Model: "mobilenetv2_1.00_160"
* Total params: 2,257,984
* Trainable params: 2,223,872
* Non-trainable params: 34,112
* # layers:  155


## Step 5 - InceptionV3 Model

Inception achieved 79% accuracy in ImageNet data

- [Reference paper](http://arxiv.org/abs/1512.00567)
- [Tensorflow implementation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/InceptionV3)


### TODO
- How many model layers
- How many model parameters
- And inspect the size of the model

In [26]:
inceptionv3_model = tf.keras.applications.InceptionV3 (input_shape=(299,299,3), 
                                                        include_top = True,
                                                        weights = 'imagenet')

print_model_summary_compact(inceptionv3_model)


* Model: "inception_v3"
* Total params: 23,851,784
* Trainable params: 23,817,352
* Non-trainable params: 34,432
* # layers:  313


In [25]:
## Model sizes
! du -skh ~/.keras/models/*

92M	/home/sujee/.keras/models/inception_v3_weights_tf_dim_ordering_tf_kernels.h5
9.0M	/home/sujee/.keras/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_160_no_top.h5
14M	/home/sujee/.keras/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5
9.0M	/home/sujee/.keras/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5


## Step 6 - Investigate other Models

You can see other pre-trained models here.  
https://www.tensorflow.org/api_docs/python/tf/keras/applications